In [1]:
#Importing packages
!pip install tweepy
!pip install wget
!pip install mysql-connector-python
import tweepy
import pandas as pd
import mysql.connector

In [2]:
#tweepy api authentication keys
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""


In [3]:
# connecting to tweepy api
auth = tweepy.OAuth1UserHandler(
   consumer_key, consumer_secret, access_token, access_token_secret
)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [4]:
# defining variables for search
new_search = "#Yelp"
num_tweets = 500
hashtag_tweets = []
user_info = []

In [5]:
#scraping twitter to get most recent tweets that have "#yelp" 
tweets = tweepy.Cursor(api.search_tweets,q=new_search,count=num_tweets,lang="en",tweet_mode='extended')
#extracting the tweet_info and user_info of all "#yelp" tweets
for tweet in tweets.items(500):
    if not tweet.entities["urls"]:
        continue
    else:
        hashtag_tweets.append((tweet.id, tweet.user.id, tweet.full_text, tweet.entities["urls"][0]["url"],tweet.retweet_count, tweet.favorite_count, tweet.created_at))
        user_info.append((tweet.user.id, tweet.user.screen_name, tweet.user.location, tweet.user.created_at))
    
    

In [6]:
#creating a dataframe to store user information
df_user = pd.DataFrame(user_info, columns =['user_id', 'username', 'location', 'created_at'])
df_user


,user_id,username,location,created_at
0,3214616837,LeanStartupLive,www.MyFrugalBusiness.com,2015-04-27 20:13:00+00:00
1,2951920127,MyFiscalFitness,www.MarketingMasterminds.org,2014-12-30 21:36:06+00:00
2,1312336063,DavidA7777,,2013-03-29 02:17:04+00:00
3,285209817,ocfooddiva,"Orange County, CA",2011-04-20 18:44:40+00:00
4,2207885690,logistikchefen,"West Palm Beach, FL",2013-11-21 21:52:06+00:00
...,...,...,...,...
455,1942769316,rsjonesfreedom,Colorado,2013-10-07 03:21:59+00:00
456,17149018,russkar,"ÜT: 21.3026949,-157.8469468",2008-11-04 04:48:44+00:00
457,29082708,richardpropes,"Indianapolis, IN",2009-04-05 22:31:03+00:00
458,54535272,cenisabelphoto,"Manteca, CA",2009-07-07 12:48:40+00:00


In [7]:
#creating a dataframe to store "#yelp" tweets information
df_hash = pd.DataFrame(hashtag_tweets, columns =['tweet_id', 'user_id', 'text', 'url','retweets','likes','time'])
df_hash

,tweet_id,user_id,text,url,retweets,likes,time
0,1590723307770646529,3214616837,RT @MyFiscalFitness: 3 Reasons To Consider Hir...,https://t.co/T438goQvCs,1,0,2022-11-10 15:09:25+00:00
1,1590723210781151232,2951920127,3 Reasons To Consider Hiring A Reputation Mana...,https://t.co/T438goQvCs,1,1,2022-11-10 15:09:02+00:00
2,1590721755433533440,1312336063,I checked in at James Coffee on #Yelp https://...,https://t.co/6pc86UWIOt,0,0,2022-11-10 15:03:15+00:00
3,1590719422846873600,285209817,Bacon World Championship starts in 7 minutes!!...,https://t.co/xukkjvqOmk,0,0,2022-11-10 14:53:59+00:00
4,1590715389570600960,2207885690,I checked in at SAS Lounge on #Yelp https://t....,https://t.co/MzcbqS8f90,0,0,2022-11-10 14:37:57+00:00
...,...,...,...,...,...,...,...
455,1589751540621144065,1942769316,Here for dinner (@ KFC) on #Yelp https://t.co/...,https://t.co/sWesVKKI60,0,0,2022-11-07 22:47:58+00:00
456,1589751013631987712,17149018,I checked in at Kapiolani Seafood Restaurant o...,https://t.co/cdbAsqCu2D,0,0,2022-11-07 22:45:52+00:00
457,1589750749902536704,29082708,I checked in at Landmark's Glendale 12 on #Yel...,https://t.co/dfKTXwJmy3,0,1,2022-11-07 22:44:49+00:00
458,1589750662656851970,54535272,I checked in at Tokyo Tei on #Yelp https://t.c...,https://t.co/Yqr90aojPv,0,0,2022-11-07 22:44:28+00:00


In [8]:
#making a list of user_id in df_user
df_user_id = df_user["user_id"].tolist()

In [9]:
recent_tweets = []

In [10]:
#extracting the timeline tweets of all users in df_user table
for user in df_user_id:
    rcnt_tweets = tweepy.Cursor(api.user_timeline,user_id=user,count=20,tweet_mode='extended')
    for tweet in rcnt_tweets.items(20):
        recent_tweets.append((tweet.id, tweet.user.id, tweet.full_text, tweet.created_at))

In [11]:
#creating a dataframe of user_timeline
df_user_tweets = pd.DataFrame(recent_tweets, columns =['tweet_id','user_id', 'text', 'created_at'])
df_user_tweets

,tweet_id,user_id,text,created_at
0,1590723402377359362,3214616837,RT @LeanStartupLive: What To Know When Working...,2022-11-10 15:09:47+00:00
1,1590723307770646529,3214616837,RT @MyFiscalFitness: 3 Reasons To Consider Hir...,2022-11-10 15:09:25+00:00
2,1590723300744830977,3214616837,RT @MikeSchiemer: The Key To Creating A Smart ...,2022-11-10 15:09:23+00:00
3,1590723284672598016,3214616837,RT @MyFiscalFitness: Delta 8 vs Delta 9 Cannab...,2022-11-10 15:09:19+00:00
4,1590699322584096768,3214616837,What To Know When Working With High Risk Merch...,2022-11-10 13:34:06+00:00
...,...,...,...,...
9161,1589352465107161090,891901724,I checked in at Scorpio Coffee on #Yelp https:...,2022-11-06 20:22:11+00:00
9162,1589140224151793670,891901724,I checked in at Rick's Dessert Diner on #Yelp ...,2022-11-06 06:18:48+00:00
9163,1589087899173421058,891901724,Smokey Robinson! (@ Hard Rock Hotel &amp; Casi...,2022-11-06 02:50:53+00:00
9164,1589062828602032129,891901724,I checked in at Azayaka Japanese Fusion on #Ye...,2022-11-06 01:11:16+00:00


In [12]:
df_user_tweets

,tweet_id,user_id,text,created_at
0,1590723402377359362,3214616837,RT @LeanStartupLive: What To Know When Working...,2022-11-10 15:09:47+00:00
1,1590723307770646529,3214616837,RT @MyFiscalFitness: 3 Reasons To Consider Hir...,2022-11-10 15:09:25+00:00
2,1590723300744830977,3214616837,RT @MikeSchiemer: The Key To Creating A Smart ...,2022-11-10 15:09:23+00:00
3,1590723284672598016,3214616837,RT @MyFiscalFitness: Delta 8 vs Delta 9 Cannab...,2022-11-10 15:09:19+00:00
4,1590699322584096768,3214616837,What To Know When Working With High Risk Merch...,2022-11-10 13:34:06+00:00
...,...,...,...,...
9161,1589352465107161090,891901724,I checked in at Scorpio Coffee on #Yelp https:...,2022-11-06 20:22:11+00:00
9162,1589140224151793670,891901724,I checked in at Rick's Dessert Diner on #Yelp ...,2022-11-06 06:18:48+00:00
9163,1589087899173421058,891901724,Smokey Robinson! (@ Hard Rock Hotel &amp; Casi...,2022-11-06 02:50:53+00:00
9164,1589062828602032129,891901724,I checked in at Azayaka Japanese Fusion on #Ye...,2022-11-06 01:11:16+00:00


In [13]:
#connect to mysql servel
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="admin",
  database="tweetdb"
)

print(mydb)

mycursor = mydb.cursor()


In [14]:
# create users table and adding contraints
mycursor.execute("DROP TABLE IF EXISTS users")
mycursor.execute("CREATE TABLE users (user_id VARCHAR(255) PRIMARY KEY, username VARCHAR(255), location VARCHAR(255), created_at TIMESTAMP)")

In [15]:
# inserting df_user dataframe in sql user table 
val = list(df_user.itertuples(index=False,name=None))
sql = "INSERT IGNORE INTO users (user_id, username, location, created_at) VALUES (%s, %s, %s, %s)"

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")

214 was inserted.


In [16]:
# create hashtag table and adding constraints
mycursor.execute("DROP TABLE IF EXISTS hashtag")
mycursor.execute("CREATE TABLE hashtag (tweet_id VARCHAR(255) PRIMARY KEY, user_id VARCHAR(255), text TEXT(6553), url VARCHAR(255), retweets INT, likes INT, time TIMESTAMP, CONSTRAINT user_id_fk FOREIGN KEY (user_id) REFERENCES users (user_id) ON UPDATE CASCADE ON DELETE CASCADE)")



In [18]:
# inserting df_hash dataframe in sql hashtag table 
val = list(df_hash.itertuples(index=False,name=None))
sql = "INSERT IGNORE INTO hashtag (tweet_id, user_id, text, url, retweets, likes, time) VALUES (%s, %s, %s, %s, %s, %s, %s)"

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")

460 was inserted.


In [19]:
# create user timeline table and adding constraints
mycursor.execute("DROP TABLE IF EXISTS timeline")
mycursor.execute("CREATE TABLE timeline (tweet_id VARCHAR(255), user_id VARCHAR(255), text TEXT(6553), created_at TIMESTAMP, CONSTRAINT past_tweets PRIMARY KEY(tweet_id,user_id), CONSTRAINT timeline_user_fk FOREIGN KEY (user_id) REFERENCES users (user_id) ON UPDATE CASCADE ON DELETE CASCADE)")

In [20]:
# inserting user_tweets dataframe in sql timeline table 
val = list(df_user_tweets.itertuples(index=False,name=None))
sql = "INSERT IGNORE INTO timeline (tweet_id, user_id, text, created_at) VALUES (%s, %s, %s, %s)"

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")

4265 was inserted.
